In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-olympiad-predictive-maintenance/test_set.csv
/kaggle/input/ml-olympiad-predictive-maintenance/sampled_submission.csv
/kaggle/input/ml-olympiad-predictive-maintenance/train_set.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import imblearn

from sklearn.decomposition import PCA

In [3]:
train_data = pd.read_csv('../input/ml-olympiad-predictive-maintenance/train_set.csv')
test_data = pd.read_csv('../input/ml-olympiad-predictive-maintenance/test_set.csv')

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61249 entries, 0 to 61248
Data columns (total 27 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   unit_nr                                61249 non-null  int64  
 1   time_cycles                            61249 non-null  int64  
 2   setting_1                              61249 non-null  float64
 3   setting_2                              61249 non-null  float64
 4   setting_3                              61249 non-null  int64  
 5   Fan inlet temperature (◦R)             61249 non-null  float64
 6   LPC outlet temperature (◦R)            61249 non-null  float64
 7   HPC outlet temperature (◦R)            61249 non-null  float64
 8   LPT outlet temperature (◦R)            61249 non-null  float64
 9   Fan inlet Pressure (psia)              61249 non-null  float64
 10  bypass-duct pressure (psia)            61249 non-null  float64
 11  HP

In [5]:
train_data.head()

,unit_nr,time_cycles,setting_1,setting_2,setting_3,Fan inlet temperature (◦R),LPC outlet temperature (◦R),HPC outlet temperature (◦R),LPT outlet temperature (◦R),Fan inlet Pressure (psia),...,Corrected fan speed (rpm),Corrected core speed (rpm),Bypass Ratio,Burner fuel-air ratio,Bleed Enthalpy,Required fan speed,Required fan conversion speed,High-pressure turbines Cool air flow,Low-pressure turbines Cool air flow,RUL
0,1,1,42.0049,0.8400,100,445.00,549.68,1343.43,1112.93,3.91,...,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670,320
1,1,2,20.0020,0.7002,100,491.19,606.07,1477.61,1237.50,9.35,...,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552,319
2,1,3,42.0038,0.8409,100,445.00,548.95,1343.12,1117.05,3.91,...,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213,318
3,1,4,42.0000,0.8400,100,445.00,548.70,1341.24,1118.03,3.91,...,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176,317
4,1,5,25.0063,0.6207,60,462.54,536.10,1255.23,1033.59,7.05,...,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754,316


In [6]:
train_data.describe()

,unit_nr,time_cycles,setting_1,setting_2,setting_3,Fan inlet temperature (◦R),LPC outlet temperature (◦R),HPC outlet temperature (◦R),LPT outlet temperature (◦R),Fan inlet Pressure (psia),...,Corrected fan speed (rpm),Corrected core speed (rpm),Bypass Ratio,Burner fuel-air ratio,Bleed Enthalpy,Required fan speed,Required fan conversion speed,High-pressure turbines Cool air flow,Low-pressure turbines Cool air flow,RUL
count,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000,...,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000,61249.000000
mean,124.325181,134.311417,23.999823,0.571347,94.031576,472.882435,579.420056,1417.896600,1201.915359,8.031626,...,2334.427590,8067.811812,9.285604,0.023252,347.760029,2228.613283,97.751396,20.864333,12.518995,133.311417
std,71.995350,89.783389,14.780722,0.310703,14.251954,26.436832,37.342647,106.167598,119.327591,3.622872,...,128.197859,85.670543,0.750374,0.004685,27.808283,145.472491,5.369424,9.936396,5.962697,89.783389
min,1.000000,1.000000,0.000000,0.000000,60.000000,445.000000,535.480000,1242.670000,1024.420000,3.910000,...,2027.570000,7845.780000,8.175700,0.020000,302.000000,1915.000000,84.930000,10.160000,6.084300,0.000000
25%,60.000000,62.000000,10.004600,0.250700,100.000000,445.000000,549.330000,1350.550000,1119.490000,3.910000,...,2387.910000,8062.630000,8.648000,0.020000,330.000000,2212.000000,100.000000,10.940000,6.566100,61.000000
50%,126.000000,123.000000,25.001400,0.700000,100.000000,462.540000,555.740000,1367.680000,1136.920000,7.050000,...,2388.060000,8083.810000,9.255600,0.020000,334.000000,2223.000000,100.000000,14.930000,8.960100,122.000000
75%,185.000000,191.000000,41.998100,0.840000,100.000000,491.190000,607.070000,1497.420000,1302.620000,10.520000,...,2388.170000,8128.350000,9.365800,0.030000,368.000000,2324.000000,100.000000,28.560000,17.135500,190.000000
max,249.000000,543.000000,42.008000,0.842000,100.000000,518.670000,644.420000,1613.000000,1440.770000,14.620000,...,2390.490000,8261.650000,11.066300,0.030000,399.000000,2388.000000,100.000000,39.890000,23.885200,542.000000


In [7]:
train_data[train_data.unit_nr == 1]

,unit_nr,time_cycles,setting_1,setting_2,setting_3,Fan inlet temperature (◦R),LPC outlet temperature (◦R),HPC outlet temperature (◦R),LPT outlet temperature (◦R),Fan inlet Pressure (psia),...,Corrected fan speed (rpm),Corrected core speed (rpm),Bypass Ratio,Burner fuel-air ratio,Bleed Enthalpy,Required fan speed,Required fan conversion speed,High-pressure turbines Cool air flow,Low-pressure turbines Cool air flow,RUL
0,1,1,42.0049,0.8400,100,445.00,549.68,1343.43,1112.93,3.91,...,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670,320
1,1,2,20.0020,0.7002,100,491.19,606.07,1477.61,1237.50,9.35,...,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552,319
2,1,3,42.0038,0.8409,100,445.00,548.95,1343.12,1117.05,3.91,...,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213,318
3,1,4,42.0000,0.8400,100,445.00,548.70,1341.24,1118.03,3.91,...,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176,317
4,1,5,25.0063,0.6207,60,462.54,536.10,1255.23,1033.59,7.05,...,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754,316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,1,317,0.0014,0.0014,100,518.67,643.45,1599.21,1416.74,14.62,...,2388.47,8184.95,8.2760,0.03,396,2388,100.00,39.58,23.6562,4
317,1,318,10.0046,0.2500,100,489.05,605.56,1509.31,1324.42,10.52,...,2388.74,8179.79,8.4656,0.03,374,2319,100.00,28.77,17.4422,3
318,1,319,10.0009,0.2520,100,489.05,605.29,1512.36,1317.11,10.52,...,2388.81,8177.70,8.4545,0.03,373,2319,100.00,28.87,17.4126,2
319,1,320,20.0060,0.7002,100,491.19,607.24,1496.88,1266.21,9.35,...,2388.87,8108.60,9.0019,0.03,368,2324,100.00,24.90,15.0215,1


In [8]:
train_data['Label'] = train_data['RUL']<=21

In [9]:
# Let's create the label (1 if RUL <= 21, 0 else)
train_data.Label.value_counts()

False    55771
True      5478
Name: Label, dtype: int64

In [10]:
# We will now create aggregated features to foollow the evolution of the sensor values on the 5 most recent cycles.
cyclesToAggregate = 5

In [11]:
# We first extract the name of the 21 sensor features for practical purposes.
sensor_features = train_data.iloc[:,5:26].columns

In [12]:
def create_aggregated_features(cyclesToAggregate : int, dataframe : pd.DataFrame):
    for sensor_feature in sensor_features:
        dataframe[f'mean_{sensor_feature}'] = 0
        dataframe[f'std_{sensor_feature}'] = 0
    for unit in train_data['unit_nr'].unique():
        for sensor_feature in sensor_features:
            dataframe.loc[dataframe['unit_nr'] == unit,f'mean_{sensor_feature}'] = dataframe.loc[dataframe['unit_nr'] == unit,f'{sensor_feature}'].rolling(5).mean()
            dataframe.loc[dataframe['unit_nr'] == unit,f'std_{sensor_feature}'] = dataframe.loc[dataframe['unit_nr'] == unit,f'{sensor_feature}'].rolling(5).std()
    return dataframe

In [13]:
new_train_data = create_aggregated_features(5, train_data)

In [14]:
for col in new_train_data.columns:
    if ('mean_' in col or 'std_' in col):
        new_train_data[col].fillna(method='backfill', inplace=True)

In [15]:
new_train_data

,unit_nr,time_cycles,setting_1,setting_2,setting_3,Fan inlet temperature (◦R),LPC outlet temperature (◦R),HPC outlet temperature (◦R),LPT outlet temperature (◦R),Fan inlet Pressure (psia),...,mean_Bleed Enthalpy,std_Bleed Enthalpy,mean_Required fan speed,std_Required fan speed,mean_Required fan conversion speed,std_Required fan conversion speed,mean_High-pressure turbines Cool air flow,std_High-pressure turbines Cool air flow,mean_Low-pressure turbines Cool air flow,std_Low-pressure turbines Cool air flow
0,1,1,42.0049,0.8400,100,445.00,549.68,1343.43,1112.93,3.91,...,330.6,19.932386,2175.0,153.222061,96.986,6.739509,14.008,5.985898,8.50730,3.575054
1,1,2,20.0020,0.7002,100,491.19,606.07,1477.61,1237.50,9.35,...,330.6,19.932386,2175.0,153.222061,96.986,6.739509,14.008,5.985898,8.50730,3.575054
2,1,3,42.0038,0.8409,100,445.00,548.95,1343.12,1117.05,3.91,...,330.6,19.932386,2175.0,153.222061,96.986,6.739509,14.008,5.985898,8.50730,3.575054
3,1,4,42.0000,0.8400,100,445.00,548.70,1341.24,1118.03,3.91,...,330.6,19.932386,2175.0,153.222061,96.986,6.739509,14.008,5.985898,8.50730,3.575054
4,1,5,25.0063,0.6207,60,462.54,536.10,1255.23,1033.59,7.05,...,330.6,19.932386,2175.0,153.222061,96.986,6.739509,14.008,5.985898,8.50730,3.575054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61244,249,251,9.9998,0.2500,100,489.05,605.33,1516.36,1315.28,10.52,...,342.8,27.123790,2196.4,166.572207,96.986,6.739509,18.028,8.465242,10.80012,5.108951
61245,249,252,0.0028,0.0015,100,518.67,643.42,1598.92,1426.77,14.62,...,355.2,35.095584,2231.6,187.920994,96.986,6.739509,23.762,11.452099,14.25470,6.937942
61246,249,253,0.0029,0.0000,100,518.67,643.68,1607.72,1430.56,14.62,...,360.6,39.373849,2244.4,197.712670,96.986,6.739509,26.740,13.560640,16.03980,8.182570
61247,249,254,35.0046,0.8400,100,449.44,555.77,1381.29,1148.18,5.48,...,361.4,38.707880,2246.6,197.322832,96.986,6.739509,27.616,12.369092,16.56468,7.466736


In [16]:
prepared_dataset = new_train_data.drop(["unit_nr"], axis = 1)

In [17]:
prepared_dataset.to_csv('../working/prepared_dataset.csv')

## Modeling

In [18]:
import sklearn
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

In [19]:
scalePosWeight = len(prepared_dataset[prepared_dataset['Label']==0])/prepared_dataset['Label'].sum()

In [20]:
XGB = XGBClassifier(scale_pos_weight=scalePosWeight, eval_metric=f1_score)

In [21]:
XGB.fit(prepared_dataset.drop(['RUL', 'Label'], axis=1), prepared_dataset.Label)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=<function f1_score at 0x7fc4e3235290>, gamma=0,
              gpu_id=-1, grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [22]:
prepared_test_set = create_aggregated_features(5, test_data)

In [23]:
for col in prepared_test_set.columns:
    if ('mean_' in col or 'std_' in col):
        prepared_test_set[col].fillna(method='backfill', inplace=True)

In [24]:
prepared_test_set

,unit_nr,time_cycles,setting_1,setting_2,setting_3,Fan inlet temperature (◦R),LPC outlet temperature (◦R),HPC outlet temperature (◦R),LPT outlet temperature (◦R),Fan inlet Pressure (psia),...,mean_Bleed Enthalpy,std_Bleed Enthalpy,mean_Required fan speed,std_Required fan speed,mean_Required fan conversion speed,std_Required fan conversion speed,mean_High-pressure turbines Cool air flow,std_High-pressure turbines Cool air flow,mean_Low-pressure turbines Cool air flow,std_Low-pressure turbines Cool air flow
0,0,1,38.4734,0.5713,100,472.8824,549.0672,1515.4699,1170.7242,12.9053,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2,31.8898,0.3428,100,480.6683,558.7492,1386.6294,1201.9154,6.8373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,3,34.6499,0.5713,100,493.0275,567.7337,1398.5125,1182.3209,10.2091,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,4,30.0495,0.5713,100,471.7574,579.4201,1440.5407,1188.1000,6.0063,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,5,31.8812,0.4365,100,473.7411,574.1691,1377.9721,1286.4247,5.7570,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54948,350,504,19.4348,0.4936,100,487.8005,558.5404,1400.1744,1192.1876,9.2901,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54949,350,505,33.1175,0.5713,100,493.8766,605.2711,1396.0510,1201.9154,8.0316,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54950,350,506,16.9021,0.5713,100,472.1954,567.1811,1457.6802,1202.8633,8.0094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54951,350,507,13.8923,0.6832,100,485.5690,585.5254,1402.5927,1145.9289,12.3842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
prepared_test_set_no_unit_nr = prepared_test_set.drop('unit_nr', axis=1)

In [26]:
predictions = XGB.predict(prepared_test_set_no_unit_nr)

In [27]:
predictions = pd.concat([prepared_test_set['unit_nr'], pd.Series(predictions)], axis=1, join='inner', names=['unit_nr', 'failure'])

In [28]:
failures_predictions = predictions.groupby('unit_nr').max().values.flatten()

In [29]:
results = pd.DataFrame({
    "unit_nr": prepared_test_set["unit_nr"].unique(),
    "failure": failures_predictions
})

In [30]:
results.to_csv("submission.csv", index = False)